<a href="https://colab.research.google.com/github/ymoslem/datasets/blob/main/Tatoeba/Tatoeba-Translations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2
!wget https://downloads.tatoeba.org/exports/sentences_base.tar.bz2

In [ ]:
!tar -xfv sentences.tar.bz2
!tar -xfv sentences_base.tar.bz2

In [ ]:
!wc -l sentences.csv
!head sentences.csv
!echo
!wc -l sentences_base.csv
!head sentences_base.csv

12449826 sentences.csv
1	cmn	我們試試看！
2	cmn	我该去睡觉了。
3	cmn	你在干什麼啊？
4	cmn	這是什麼啊？
5	cmn	今天是６月１８号，也是Muiriel的生日！
6	cmn	生日快乐，Muiriel！
7	cmn	Muiriel现在20岁了。
8	cmn	密码是"Muiriel"。
9	cmn	我很快就會回來。
10	cmn	我不知道。

12449826 sentences_base.csv
1	\N
2	\N
3	\N
4	\N
5	\N
6	\N
7	\N
8	\N
9	\N
10	\N


In [ ]:
!pip3 install polars datasets -q

In [ ]:
from polars import Config

Config.set_fmt_str_lengths(1000)

In [ ]:
import polars as pl

df_sents = pl.read_csv("sentences.csv",
                        has_header=False,
                        separator="\t",
                        quote_char="",
                        null_values="\\N",
                        new_columns=["ID", "Lang", "Sentence"])

print(df_sents.shape)

# Drop nulls
df_sents = df_sents.drop_nulls()
print(df_sents.shape)

df_sents.head()

(12449826, 3)
(12449352, 3)


ID,Lang,Sentence
i64,str,str
1,"""cmn""","""我們試試看！"""
2,"""cmn""","""我该去睡觉了。"""
3,"""cmn""","""你在干什麼啊？"""
4,"""cmn""","""這是什麼啊？"""
5,"""cmn""","""今天是６月１８号，也是Muiriel的生日！"""


In [ ]:
df_mapping = pl.read_csv("sentences_base.csv",
                         has_header=False,
                         separator="\t",
                         quote_char="",
                         null_values="\\N",
                         new_columns=["ID", "id_src"])

print(df_mapping.shape)

# Drop nulls
df_mapping = df_mapping.drop_nulls()

# Convert to Int
df_mapping = df_mapping.with_columns(
                          pl.col("id_src").cast(pl.Int64)
                        )
print(df_mapping.shape)

df_mapping = df_mapping.filter(
                          pl.col("id_src") > 0
                        )

print(df_mapping.shape)

df_mapping

(12449826, 2)
(12121285, 2)
(8706605, 2)


ID,id_src
i64,i64
330930,270958
330931,138554
330932,56531
330933,56532
330934,260636
…,…
12910360,8433336
12910361,12909955
12910362,9518807


In [ ]:
# Join df_sents and df_mapping on the ID column
combined = df_sents.join(df_mapping, on="ID").drop_nulls()

combined = combined.rename({"ID": "id_tgt",
                            "Lang": "lang_tgt",
                            "Sentence": "sentence_tgt"})
print(combined.shape)
combined.head()

(8706319, 4)


id_tgt,lang_tgt,sentence_tgt,id_src
i64,str,str,i64
330930,"""fra""","""Rien ne vaut le sommeil.""",270958
330931,"""fra""","""Essayez de faire preuve de patience avec les autres.""",138554
330932,"""fra""","""Tous ces œufs ne sont pas frais.""",56531
330933,"""fra""","""Aucun de ces œufs n'est frais.""",56532
330934,"""fra""","""On m'a nommé d'après lui.""",260636


In [ ]:
# Perform a self-join to fetch the translation sentence
df_translations = combined.join(df_sents.rename({"ID": "id_src",
                                                 "Lang": "lang_src",
                                                 "Sentence": "sentence_src"}),
                                left_on="id_src",
                                right_on="id_src",
                                how="left"
                            )
print(df_translations.shape)

df_translations = df_translations.drop_nulls()
print(df_translations.shape)

df_translations.head()

(8706319, 6)
(8651840, 6)


id_tgt,lang_tgt,sentence_tgt,id_src,lang_src,sentence_src
i64,str,str,i64,str,str
330930,"""fra""","""Rien ne vaut le sommeil.""",270958,"""eng""","""There is nothing like sleep."""
330931,"""fra""","""Essayez de faire preuve de patience avec les autres.""",138554,"""jpn""","""他人には辛抱するようにしなさい。"""
330932,"""fra""","""Tous ces œufs ne sont pas frais.""",56531,"""eng""","""Not all of these eggs are fresh."""
330933,"""fra""","""Aucun de ces œufs n'est frais.""",56532,"""eng""","""None of these eggs are fresh."""
330934,"""fra""","""On m'a nommé d'après lui.""",260636,"""eng""","""My name was called after his name."""


In [ ]:
df_translations = df_translations.unique().sort("lang_src")
print(df_translations.shape)

df_translations = df_translations.select(
    [
    "id_src", "lang_src", "sentence_src",  # Source columns
    "id_tgt", "lang_tgt", "sentence_tgt"  # Target columns
]
)
df_translations

(8651840, 6)


id_src,lang_src,sentence_src,id_tgt,lang_tgt,sentence_tgt
i64,str,str,i64,str,str
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894930,"""eng""","""There’s no psychological harm."""
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894928,"""eng""","""There is no psychological damage."""
2481698,"""abk""","""Сара ашә сфоит.""",4110864,"""yue""","""我食芝士。"""
2481698,"""abk""","""Сара ашә сфоит.""",3863883,"""ita""","""Mangio formaggio."""
499353,"""acm""","""واحد اسمه هنري كال يريد يشوفك.""",3013431,"""tur""","""Adı Henry olan biri sizi görmek istedi."""
…,…,…,…,…,…
12160925,"""zza""","""Mi xo rê vizêrî komputerêka newîye herînaye.""",12330372,"""kmr""","""Min do ji xwe re kompîtureke nû kirî."""
9144120,"""zza""","""Ti senîn î?""",9244449,"""zza""","""Ti senîn a?"""
9144133,"""zza""","""Ti kamca de bîyî?""",9244408,"""zza""","""Ti kamca de bîya?"""


In [ ]:
# Remove rows with the same language
df_translations = df_translations.filter(
    (pl.col("lang_src") != pl.col("lang_tgt"))
)
print(df_translations.shape)

df_translations

(8547819, 6)


id_src,lang_src,sentence_src,id_tgt,lang_tgt,sentence_tgt
i64,str,str,i64,str,str
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894930,"""eng""","""There’s no psychological harm."""
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894928,"""eng""","""There is no psychological damage."""
2481698,"""abk""","""Сара ашә сфоит.""",4110864,"""yue""","""我食芝士。"""
2481698,"""abk""","""Сара ашә сфоит.""",3863883,"""ita""","""Mangio formaggio."""
499353,"""acm""","""واحد اسمه هنري كال يريد يشوفك.""",3013431,"""tur""","""Adı Henry olan biri sizi görmek istedi."""
…,…,…,…,…,…
11891630,"""zza""","""Embaza mi goşt ra hes kena.""",11915379,"""tur""","""Arkadaşım eti sever."""
6096269,"""zza""","""Bejné lajekî dergo.""",8140003,"""tur""","""Çocuğun boyu uzundur."""
12160925,"""zza""","""Mi xo rê vizêrî komputerêka newîye herînaye.""",12330372,"""kmr""","""Min do ji xwe re kompîtureke nû kirî."""


In [ ]:
# Create a lang_pair column and sort it to be able to filter based on it later

df_translations = df_translations.with_columns(
    pl.concat_list([pl.col("lang_src"), pl.col("lang_tgt")]).alias("lang_pair")
)

df_translations = df_translations.with_columns(
    pl.col("lang_pair").list.sort()
)

df_translations

id_src,lang_src,sentence_src,id_tgt,lang_tgt,sentence_tgt,lang_pair
i64,str,str,i64,str,str,list[str]
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894930,"""eng""","""There’s no psychological harm.""","[""abk"", ""eng""]"
12894830,"""abk""","""Апсихологиатә ԥхасҭа ыҟаӡам.""",12894928,"""eng""","""There is no psychological damage.""","[""abk"", ""eng""]"
2481698,"""abk""","""Сара ашә сфоит.""",4110864,"""yue""","""我食芝士。""","[""abk"", ""yue""]"
2481698,"""abk""","""Сара ашә сфоит.""",3863883,"""ita""","""Mangio formaggio.""","[""abk"", ""ita""]"
499353,"""acm""","""واحد اسمه هنري كال يريد يشوفك.""",3013431,"""tur""","""Adı Henry olan biri sizi görmek istedi.""","[""acm"", ""tur""]"
…,…,…,…,…,…,…
11891630,"""zza""","""Embaza mi goşt ra hes kena.""",11915379,"""tur""","""Arkadaşım eti sever.""","[""tur"", ""zza""]"
6096269,"""zza""","""Bejné lajekî dergo.""",8140003,"""tur""","""Çocuğun boyu uzundur.""","[""tur"", ""zza""]"
12160925,"""zza""","""Mi xo rê vizêrî komputerêka newîye herînaye.""",12330372,"""kmr""","""Min do ji xwe re kompîtureke nû kirî.""","[""kmr"", ""zza""]"


# Upload to HF Hub

In [ ]:
from datasets import Dataset

data_dict = df_translations.to_dict(as_series=False)

dataset = Dataset.from_dict(data_dict)

dataset

Dataset({
    features: ['id_src', 'lang_src', 'sentence_src', 'id_tgt', 'lang_tgt', 'sentence_tgt', 'lang_pair'],
    num_rows: 8547819
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
commit_message = "Tatoeba Translations"
commit_description="Tatoeba Multilingual Translations"

dataset.push_to_hub("ymoslem/Tatoeba-Translations",
                    data_dir="data",
                    private=True,
                    commit_message=commit_message,
                    commit_description=commit_description,
                    )